In [ ]:
import math
import torch
import gpytorch 

class ExactGPmodel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y,likelihood):
        super(ExactGPmodel, self).__init__(train_x, train_y,likelihood)
        num_features = train_x.size(1)
        self.mean_module = gpytorch.means.ZeroMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel(ard_num_dims=num_features))

        
    def forward(self,x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)
        
def buildModel(Xtrain,ytrain,ini=None):
    # number of features and number of training pairs
    [numData,numFeatures] = Xtrain.size()
    N = ytrain.size()[1]
    
    # initialize likelihood and model
    likelihoods = []
    models = []

    # instancelize likelihood and model
    for n in range(N):
        likelihood = gpytorch.likelihoods.GaussianLikelihood()
        likelihood.noise_covar.noise = ytrain[:,n].std()
        
        model = ExactGPmodel(Xtrain, ytrain[:,n], likelihood) 
        if ini is None:
            ini = torch.abs(0*torch.randn([1,numFeatures])+10)
        model.covar_module.base_kernel.lengthscale = ini
        model.covar_module.outputscale = ytrain[:,n].std()
        
        likelihoods.append(likelihood)
        models.append(model)
        
    return likelihoods, models